In [40]:
import robotic as ry
import numpy as np
import transformations as tf
import time
import requests
import xml.etree.ElementTree as ET
from scipy.spatial.transform import Rotation as R

In [1]:
def get_frameStructure(file_path):
    # Create a dictionary to store the child-parent frame mapping
    frame_dict = {}
    
    # Parse the URDF file
    tree = ET.parse(file_path)
    root = tree.getroot()
    frame_dict['world'] = 'world' # Add the world frame manually to this dictionary to avoid future problems

    # Iterate over each joint element in the URDF
    for joint in root.findall('joint'):
        # Get the parent and child frames
        parent_frame = joint.find('parent').attrib['link']
        child_frame = joint.find('child').attrib['link']
        
        # Add the child-parent pair to the dictionary
        frame_dict[child_frame] = parent_frame
    
    return frame_dict

def getrelativeQuat(quat_A, quat_B):
    # Convert quaternions from WXYZ to XYZW
    quat_A_xyzw = [quat_A[1], quat_A[2], quat_A[3], quat_A[0]]  # Convert to XYZW
    quat_B_xyzw = [quat_B[1], quat_B[2], quat_B[3], quat_B[0]]  # Convert to XYZW
    
    # Create rotation objects for both quaternions
    rotation_A = R.from_quat(quat_A_xyzw)  # Quaternion format is (x, y, z, w)
    rotation_B = R.from_quat(quat_B_xyzw)

    # Calculate the inverse of rotation_A
    inverse_A = rotation_A.inv()

    # Multiply rotation_B by inverse_A to get the relative rotation
    relative_rotation = inverse_A * rotation_B

    # Get the relative quaternion in XYZW format
    relative_quat = relative_rotation.as_quat()

    # Return in WXYZ format
    return [relative_quat[3], relative_quat[0], relative_quat[1], relative_quat[2]]  # Convert back to WXYZ

def get_toSendData(frameNames, frameState, parent_dict):
    # Prepares the data in form of dictionary where key is the frame name and value is a 6-valued list: P P P R R R where P is the absolute position co-ordinate in world frame, 
    # while R is the relative rotation around a certain axis from parent to current frame  
    FramePoses = {}
    for FrameName in frameNames:
            # Eliminate the framenames with _0 extension or whatever. This assumes that all the frames are taken into account in parent_dict dictionary!
            if FrameName not in parent_dict:
                    continue
            
            # Append "_0" to Child_Frame and Parent_Frame to get correct frame state
            Child_Frame = FrameName + "_0"
            Parent_Frame = parent_dict[FrameName] + "_0"
            
            if Parent_Frame not in frameNames:
                Parent_Frame = parent_dict[FrameName]

            if Child_Frame not in frameNames:
                Child_Frame = FrameName 
                
            Child_State = frameState[frameNames.index(Child_Frame)]
            Parent_State = frameState[frameNames.index(Parent_Frame)]
            toSend = QuatToDOFConv(Child_State, Parent_State)
            FramePoses[FrameName] = toSend.tolist()
            
    return FramePoses


def QuatToDOFConv(child_state, parent_state):
    # Converts the data from Quaternion to Euler Angles
    child_quat = np.array(child_state)
    parent_quat = np.array(parent_state)

    # Separate translation and quaternion
    child_translation = child_quat[:3]  # The first 3 values are the translation
    parent_translation = parent_quat[:3]  # The first 3 values are the translation  
    child_quaternion = child_quat[3:]   # The last 4 values are the quaternion 
    parent_quaternion = parent_quat[3:]
    
    # Calculate the relative Quaternion from the parent to child
    parentChild_RelQuat =  getrelativeQuat(parent_quaternion, child_quaternion)
    euler_angles = tf.euler_from_quaternion(parentChild_RelQuat, axes='sxyz')  # 'sxyz' is a common axis convention
    euler_angles_deg = np.degrees(euler_angles)
    # Combine translation and rotation into a 6 DOF representation
    
    #X Y Z W
    six_dof = np.hstack((child_translation, euler_angles_deg))

    return six_dof

def pickObject(gripperframeName, objectframeName, Config):
    # A function that uses Inverse Kinematics to move the gripper to object and then attaches frame gripperframeName to frame objectframeName
    komo = ry.KOMO(Config, 1, 40, 2, False)
    komo.addControlObjective([], 0, 1e-1) # what happens if you change weighting to 1e0? why?
    komo.addControlObjective([], 1, 1e0)
    komo.addObjective([1], ry.FS.positionDiff, [gripperframeName, objectframeName], ry.OT.eq, [1e1])

    ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
    print(ret)
    q = komo.getPath()
    print('size of path:', q.shape)

    for t in range(q.shape[0]):
        Config.setJointState(q[t])
        Config.view(False, f'waypoint {t}')
        Fstate = Config.getFrameState()
        Fnames = Config.getFrameNames()
        toSend = get_toSendData(Fnames, Fstate, parent_dict)
        POSTdata(toSend)
        time.sleep(.1)
    
    # Attach the object to gripper
    Config.attach(gripperframeName, objectframeName)

def placeObject(gripperframeName, objectframeName, desiredLocation, Config):
    # A function that uses Inverse Kinematics to move the compound gripper-object frame to desiredLocation frame and detaches the gripper from object over there
    komo = ry.KOMO(Config, 1, 40, 2, False)
    komo.addControlObjective([], 0, 1e-1) # what happens if you change weighting to 1e0? why?
    komo.addControlObjective([], 1, 1e0)
    komo.addObjective([1], ry.FS.positionDiff, [gripperframeName, desiredLocation], ry.OT.eq, [1e1])

    ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
    print(ret)
    q = komo.getPath()
    print('size of path:', q.shape)

    for t in range(q.shape[0]):
        Config.setJointState(q[t])
        Config.view(False, f'waypoint {t}')
        Fstate = Config.getFrameState()
        Fnames = Config.getFrameNames()
        toSend = get_toSendData(Fnames, Fstate, parent_dict)
        POSTdata(toSend)
        time.sleep(.1)
    # Detach the object from gripper
    Config.getFrame(objectframeName).unLink()

def moveRobot(gripperframeName, desiredLocation, Config):
    # A function that uses Inverse Kinematics to move the frame gripperframeName to frame desiredLocation 
    komo = ry.KOMO(Config, 1, 40, 2, False)
    komo.addControlObjective([], 0, 1e-1) # what happens if you change weighting to 1e0? why?
    komo.addControlObjective([], 1, 1e0)
    komo.addObjective([1], ry.FS.positionDiff, [gripperframeName, desiredLocation], ry.OT.eq, [1e1])

    ret = ry.NLP_Solver(komo.nlp(), verbose=0 ) .solve()
    print(ret)
    q = komo.getPath()
    print('size of path:', q.shape)

    for t in range(q.shape[0]):
        Config.setJointState(q[t])
        Config.view(False, f'waypoint {t}')
        Fstate = Config.getFrameState()
        Fnames = Config.getFrameNames()
        toSend = get_toSendData(Fnames, Fstate, parent_dict)
        POSTdata(toSend)
        time.sleep(.1)


# Will just take the data and post it without worrying about how the data looks like and will not seek any confirmation from other side
def POSTdata(FramePoses):
    response = requests.post('http://localhost:8000/coordinates', json=FramePoses) # Change this localhost thing when using Windows for VR!


In [77]:
C = ry.Config()
C.addFile(ry.raiPath('Tiago/Tiago_SimpleScene.g'))
C.addFrame('way1'). setShape(ry.ST.marker, [.3]) .setPosition([0, -1.5, 0.9])
C.addFrame('way2'). setShape(ry.ST.marker, [.3]) .setPosition([3.8, .4, 0.9])
qHome = C.getJointState()
fHome = C.getFrameState()
C.view()
        
# Get the Parent_Child Relationship for all frames, read from a URDF file. parent_dict['current_frame'] = 'parent_of_current_frame'
urdf_file = ry.raiPath('Tiago/Tiago_SimpleScene.urdf')
parent_dict = get_frameStructure(urdf_file)

In [88]:
# Move the robot to the block_01 by entering the desired block frame!
desiredFrameName = 'way1'

# Move the object back to its starting location before initiating any motion planning
C.setJointState(qHome)
C.setFrameState(fHome)
C.view()
gripper = 'right_inner_knuckle'
objectPick_01 = 'Block_01_0' # Always add _0 for correct frame to be manipulated!
objectPick_02 = 'Block_02_0' # Always add _0 for correct frame to be manipulated!

# This set of commands make the robot to pick the object-01 and place it on way1, then pick the object-02 and place it on way2 and then move the robot back!
pickObject(gripper, objectPick_01, C)
placeObject(gripper, objectPick_01, 'way1', C)
pickObject(gripper, objectPick_02, C)
placeObject(gripper, objectPick_02, 'way2', C)
moveRobot(gripper, 'world', C)

# Get the final Frames and Joints State
qLast = C.getJointState()
fLast = C.getFrameState()

{ time: 0.019847, evals: 26, done: 1, feasible: 1, sos: 10.2399, f: 0, ineq: 0, eq: 7.92799e-05 }
size of path: (40, 21)
{ time: 0.018327, evals: 24, done: 1, feasible: 1, sos: 10.6074, f: 0, ineq: 0, eq: 0.000101526 }
size of path: (40, 21)
{ time: 0.010808, evals: 22, done: 1, feasible: 1, sos: 7.6078, f: 0, ineq: 0, eq: 9.77986e-05 }
size of path: (40, 21)
{ time: 0.021757, evals: 38, done: 1, feasible: 1, sos: 31.1974, f: 0, ineq: 0, eq: 0.000121617 }
size of path: (40, 21)
{ time: 0.018509, evals: 27, done: 1, feasible: 1, sos: 12.8152, f: 0, ineq: 0, eq: 0.000271312 }
size of path: (40, 21)


In [39]:
# Clean delete every single jupyter environment entity
for var in list(globals().keys()):
    del globals()[var]

del var